In [2]:
import cv2
import numpy as np
from dataloader import Trial
# import matplotlib.pyplot as plt

In [3]:
video_path = r'..\data\TH8\TH8ST17A3D250TS0_2.mov'

In [47]:
recording = Trial(video_path)
recording.to_gray(inplace=True)
recording.change_contrast(alpha=1.25, beta=-1, inplace=True)
recording.gauss_blur(ksize=(5,5), sigmaX=1, inplace=True, sigmaY=1)
recording.gauss_blur(ksize=(5,5), sigmaX=1, inplace=True, sigmaY=1)
# recording.sharpen(inplace=True)
recording.gauss_blur(ksize=(3,3), sigmaX=1, inplace=True, sigmaY=1)
recording.gauss_blur(ksize=(3,3), sigmaX=1, inplace=True, sigmaY=1)

# recording.dilate(inplace=True, ksize=(3,3))
recording.erode(inplace=True, ksize=(3,3))

In [50]:
out = recording.edges(thresh1=20, thresh2=50)
# out.dilate(inplace=True, ksize=(5,5))
# out.erode(inplace=True, ksize=(5,5))
# out = out.create_overlay(other=Trial(video_path))
# out = out.create_overlay(other=Trial(video_path))
out.play_video()

In [42]:
out.play_video()

In [32]:
recording.play_video()

In [10]:
recording.edges(thresh1=20, thresh2=120, inplace=True)

In [ ]:
overlayed = recording.create_overlay()
overlayed.play_video()

In [25]:
ret, labels = cv2.connectedComponents(recording[0])
normalized_labels = labels / ret
label_hue = np.uint8(179 * normalized_labels)
blank_ch = np.uint8(255*np.ones_like(label_hue))
labeled_img_hsv = cv2.merge([label_hue, blank_ch, blank_ch])
print(labeled_img_hsv.shape)
labeled_img_bgr = cv2.cvtColor(labeled_img_hsv, cv2.COLOR_HSV2BGR)
labeled_img_bgr[label_hue==0] = 0
cv2.imshow("Test", labeled_img_bgr)
cv2.waitKey(0)
cv2.destroyAllWindows()
# recording[0]
# tempo
# ret

(1008, 1264, 3)
